<img src="logo.png" alt="Logo" width="10%">

#    Supervivencia en el Titanic: Una historia contada con datos

## 1. Introducción

El hundimiento del RMS Titanic es uno de los naufragios más famosos de la historia. El 15 de abril de 1912, durante su viaje inaugural, el Titanic se hundió tras chocar con un iceberg, resultando en la muerte de 1.502 personas de las 2.224 que viajaban a bordo.

Este análisis busca responder a la pregunta: **¿Qué factores influyeron en la supervivencia de los pasajeros?** Exploraremos variables como clase social, género, edad y más para entender patrones que podrían haber determinado quién sobrevivió a esta tragedia.

A través de este análisis, no solo estaremos aplicando técnicas de ciencia de datos, sino también contando la historia humana detrás de los números, honrando la memoria de quienes estuvieron en esta tragedia.

## 2. Preparación de los Datos

Primero, carguemos las bibliotecas necesarias y exploremos el conjunto de datos:

In [ ]:
# Importamos las bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuraciones para visualizaciones
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
sns.set(font_scale=1.2)

# Para mostrar todas las columnas del DataFrame
pd.set_option('display.max_columns', None)

# Para evitar advertencias
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Cargamos el dataset
df = pd.read_csv('df_titanic.csv')

# Visualizamos las primeras filas
df.head()

### 2.1 Explorando el Dataset

Nuestro dataset contiene información detallada sobre los pasajeros del Titanic. Veamos qué información tenemos disponible y su estructura:

In [ ]:
# Verificamos el tamaño del dataset
print(f"Dimensiones del dataset: {df.shape}")

# Verificamos los tipos de datos
print("\nTipos de datos:")
print(df.dtypes)

# Verificamos valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

### 2.2 Descripción de Columnas

Nuestro dataset contiene la siguiente información:

- **pclass**: Clase del pasajero (1 = Primera clase, 2 = Segunda clase, 3 = Tercera clase)
- **survived**: Si el pasajero sobrevivió (1) o no (0)
- **name**: Nombre del pasajero
- **sex**: Género del pasajero
- **age**: Edad del pasajero
- **sibsp**: Número de hermanos/cónyuges a bordo
- **parch**: Número de padres/hijos a bordo
- **ticket**: Número de boleto
- **fare**: Tarifa pagada
- **cabin**: Número de cabina
- **embarked**: Puerto de embarque (C = Cherbourg, Q = Queenstown, S = Southampton)
- **boat**: Bote salvavidas (si sobrevivió)
- **body**: Número de identificación del cuerpo (si no sobrevivió y fue recuperado)
- **home.dest**: Destino del hogar

Observamos que hay valores nulos en varias columnas, especialmente en 'age', 'cabin', 'boat', 'body' y 'home.dest'.

In [ ]:
# Estadísticas descriptivas
df.describe()

### 2.3 Análisis Estadístico Preliminar

A partir de las estadísticas descriptivas podemos observar que:

- La tasa de supervivencia general fue aproximadamente del 38.2%.
- La edad promedio de los pasajeros era de aproximadamente 29.9 años, con personas desde bebés hasta ancianos de 80 años.
- La mayoría de los pasajeros viajaban solos o con pocos familiares (el promedio de sibsp es 0.5 y de parch es 0.38).
- Las tarifas variaban enormemente, desde pasajeros que no pagaron hasta aquellos que pagaron más de 500 libras esterlinas.
- La mayoría de los pasajeros viajaba en tercera clase (aproximadamente el 55%).

## 3. Limpieza y Preparación de Datos

Antes de avanzar con el análisis exploratorio, necesitamos limpiar los datos y manejar los valores faltantes.

In [ ]:
# Crear una copia del DataFrame para no modificar el original
df_cleaned = df.copy()

# Verificar la cantidad de valores nulos en cada columna
missing_values = df_cleaned.isnull().sum().sort_values(ascending=False)
percent_missing = (df_cleaned.isnull().sum() / len(df_cleaned) * 100).sort_values(ascending=False)

missing_data = pd.concat([missing_values, percent_missing], axis=1, keys=['Total', 'Porcentaje'])
missing_data

### 3.1 Manejo de Valores Nulos

Podemos observar que tenemos valores nulos en varias columnas. Vamos a manejarlos de la siguiente manera:

1. **Age (Edad)**: Imputaremos los valores faltantes con la mediana según la clase y el género.
2. **Cabin (Cabina)**: Tiene muchos valores nulos (más del 77%). Crearemos una nueva variable para indicar si se conoce o no la cabina.
3. **Embarked (Puerto de embarque)**: Solo tiene 3 valores nulos, los imputaremos con el valor más frecuente.
4. **Fare (Tarifa)**: Solo tiene 2 valores nulos, los imputaremos con la mediana según la clase.
5. **Otras columnas**: Las columnas como 'boat', 'body' y 'home.dest' tienen relación con el resultado (supervivencia), así que no las imputaremos.